In [124]:
import pandas as pd
import numpy as np
from datetime import datetime
import numpy_financial as npf

In [125]:
#Reading data from excel
raw_data = pd.read_excel("../test.xlsx")

In [95]:
# Parsing the dates and keeping null incase it's unparsable
# date_columns = ["Disbursal date"]
# for col in date_columns:
#     raw_data[col] = pd.to_datetime(raw_data[col], errors="coerce")

In [126]:
# #adding 0 values for missing numerical data
# raw_data.fillna({
#     "Loan amount": 0,
#     "Days late": 0,
#     "Monthly payment": 0,
#     "Outstanding principal": 0,
#     "Repaid principal": 0,
#     "Outstanding interest": 0,
#     "Repaid interest": 0,
#     "DTI": 0,
#     "Borrower income": 0,
#     "Annual profit": 0,
#     "Arrears": 0
# }, inplace=True)

In [97]:
# #Separating Raw data based on borrower type
# allowed_borrower_types = ["individual", "business"]

# valid_df = raw_data[raw_data["Borrower type"].isin(allowed_borrower_types)]
# if len(valid_df) != len(raw_data):
#     invalid_df = raw_data[~raw_data["Borrower type"].isin(allowed_borrower_types)]
#     print("These data have invalid borrower type", invalid_df)

# table_individual = valid_df[valid_df['Borrower type']=="individual"]
# table_business = valid_df[valid_df['Borrower type']=="business"]


In [98]:
#Preprocessing Data based on borrower type

# columns_required_for_both = [
# "Borrower ID", "Loan ID", "Credit score", "Loan amount", "Disbursal date", "Interest rate", "Loan term", "Borrower type", "Loan type", 
# "Expected repayment date", "Loan status", "Days late", "Monthly payment", "Outstanding principal", "Repaid principal", "payments", "Outstanding interest", 
# "Repaid interest", "Repayment date", "Delay interest", "Arrears"
# ]
# columns_required_for_individual= columns_required_for_both + ["Employment status", "DTI", "Borrower income", "Last debt payment date"]
# columns_required_for_business= columns_required_for_both + ["Annual profit"]

In [99]:
# table_individual = valid_df[columns_required_for_individual]
# table_individual = valid_df[columns_required_for_business]

In [ ]:
def detect_loan_anomalies(df: pd.DataFrame):
    anomalies_dict = {}

    for _, loan in df.iterrows():
        loan_id = loan["Loan ID"]
        anomalies_dict[loan_id] = set()

        # Data validation anomalities
        if loan["Interest rate"] < 0 or loan["Interest rate"] > 100:
            anomalies_dict[loan_id].add("Invalid interest rate")

        if loan["Loan term"] <= 0:
            anomalies_dict[loan_id].add("Loan term must be positive non-zero value")

        if loan["Credit score"] not in ["A", "B", "C", "D"]:
            anomalies_dict[loan_id].add("Credit score not in range")
        
        if loan["Loan status"] == "repaid" and loan["Outstanding principal"] > 0:
            anomalies_dict[loan_id].add("Outstanding principal must be 0 when loan status is repaid")
        
        if loan["Loan amount"] < 0:
            anomalies_dict[loan_id].add("Loan amount must be positive")
        
        # Detecting repayment date anomaly
        days_late = loan["Days late"]
        if days_late > 90:
            anomalies_dict[loan_id].add(f"Re-payment date anomaly as {days_late} is greater than 90 days threshold")

        #Detecting payments history anomaly
        payments = loan["payments"]
        if isinstance(payments, str):
            try:
                payments = eval(payments)
                #Checking if most of the payments state are pending late or paid with delay
                threshold = 0.5
                matching_count = sum(payment['State'] in ['pending late', 'paid with delay'] for payment in payments)
                if matching_count/len(payments) > threshold:
                    anomalies_dict[loan_id].add(f"Payments history are not consistent")
                
            except Exception as e:
                    # print(f"Payment history parsing error {e} for {loan_id}")
                    anomalies_dict[loan_id].add(f"Payment history parsing error {e}")

        #Checking if expected repayment date was earlier than actual repayment date
        expected_date = pd.to_datetime(loan['Expected repayment date'], errors='coerce')
        repayment_date = pd.to_datetime(loan['Repayment date'], errors='coerce')    
        if pd.notna(expected_date) and pd.notna(repayment_date):
            date_dif = repayment_date-expected_date
            if date_dif.days > 90:
                anomalies_dict[loan_id].add(f"90 days crossed between expected repayment date and actual repayment date")

    return anomalies_dict

anomalies = detect_loan_anomalies(raw_data)
# print(anomalies)
# print(anomalies[anomalies["Anomalies"] != "Normal"])
count = 0
all_results = []
for loan_id, anomalies in anomalies.items():
    if not anomalies:
        count += 1
    all_results.append({
        'Loan ID': loan_id, 
        'Anomalies': ", ".join(anomalies) if anomalies else "Normal"
    })
anomaly_results = pd.DataFrame(all_results)

print(f"{count=}")
# print((anomaly_results[anomaly_results["Anomalies"] != "Normal"]))
anomaly_results.to_excel("Output.xlsx")


count=42


In [ ]:
def calculate_xirr(cashflows, dates):
    """Calculate XIRR for given cashflows and dates"""
    date_deltas = [(date - dates[0]).days for date in dates]
    rate = npf.xirr(cashflows, date_deltas)
    return rate